# TO DO
1. we have webscrape the "metacrtics"  website for top 500 movies 

# Task 1 : Webscraping
 import required libaries 
    * requests : To retrived the Html of the source page
    * bs4      : To converted the obtained html which in the form of text into soup object in which we can retrive specific                      data like headers, paragraphs, etc.
    * lxml     : Helps bs4 to understand the html.

In [3]:
import requests
import lxml
import bs4


In [4]:
urls =["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=0",
      "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1",
      "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2",
      "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3",
      "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4"]



In [ ]:
# movie_short_href={}
# movie_details={}
# failed_items=[]

#Uncomment and run for the first time


In [5]:

index_num=0
for url in urls:
    result = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(result.text, "lxml")
    div_tag = soup.find_all('div', class_=["browse_list_wrapper one browse-list-large",
                                           "browse_list_wrapper two browse-list-large",
                                           "browse_list_wrapper three browse-list-large",
                                           "browse_list_wrapper four browse-list-large"])
    for item in div_tag:
        for element in item.find_all('td', class_='clamp-image-wrap'):
            index_num=index_num+1
            source=element.find_all('a' , href= True)[0]
            movie_short_href[index_num]=source['href'].replace('/movie/','')
    
    

In [60]:

movie_short_href




{1: 'citizen-kane',
 2: 'the-godfather',
 3: 'rear-window',
 4: 'casablanca',
 5: 'boyhood',
 6: 'three-colors-red',
 7: 'vertigo',
 8: 'notorious-1946',
 9: 'singin-in-the-rain',
 10: 'city-lights',
 11: 'moonlight-2016',
 12: 'intolerance',
 13: 'pinocchio-1940',
 14: 'touch-of-evil',
 15: 'the-lady-vanishes-1938',
 16: 'the-treasure-of-the-sierra-madre',
 17: 'pans-labyrinth',
 18: 'some-like-it-hot',
 19: 'north-by-northwest',
 20: 'hoop-dreams',
 21: 'rashomon',
 22: 'all-about-eve',
 23: 'jules-and-jim',
 24: 'the-wild-bunch',
 25: 'my-left-foot',
 26: 'the-third-man',
 27: 'dr-strangelove-or-how-i-learned-to-stop-worrying-and-love-the-bomb',
 28: 'quo-vadis-aida',
 29: 'gone-with-the-wind',
 30: '4-months-3-weeks-and-2-days',
 31: 'psycho-1960',
 32: 'battleship-potemkin',
 33: 'a-streetcar-named-desire',
 34: 'american-graffiti',
 35: 'dumbo-1941',
 36: 'roma',
 37: 'ran',
 38: 'the-shop-around-the-corner',
 39: '12-angry-men',
 40: 'manchester-by-the-sea',
 41: 'rosemarys-baby

In [61]:
def retrive_data_from_website(i):
    movie_link='https://www.metacritic.com/movie/'
    url = movie_link+movie_short_href[i]+'/details'
    result = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    if result.status_code != 200:
        failed_items.append(i)
    else:
        soup = bs4.BeautifulSoup(result.text,'lxml')
        tags = soup.find_all('table', class_='credits')
        wanted_columns=['Director', 'Principal Cast', 'Cast']
        final_details={}
        for table in tags:
            column = table.find_all('th', class_="person")[0].getText()
            if column in wanted_columns:
                details=[]
                for name in table.find_all('a', href=True):
                    details.append(name.getText().strip())
                final_details[column]=details
    if i in failed_items:
        failed_items.remove(i)
    movie_details[movie_short_href[i]]=final_details

In [57]:


    
for i in movie_short_href.keys():
    retrive_data_from_website(i)
        


    

In [63]:
print(failed_items)


[1, 26, 94, 199, 200, 312, 313, 334, 400, 422]


In [62]:
if len(failed_items)>0:
    for i in failed_items:
        retrive_data_from_website(i)